In [ ]:
from pathlib import Path
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1 import make_axes_locatable
import h5py
import copy
import pickle

# Project modules
from lib.Stokes import Stokes
from functions.plot_data import plot_data
from functions.plot_angle_gradient import plot_angle_gradient

In [ ]:
# Open data files
from functions.load_pickles import load_pickles
stokes_list, derived = load_pickles()

In [ ]:
# Extract each Stokes parameter into dictionary, to make it easier to work with
I = stokes_list['I']
Q = stokes_list['Q']
U = stokes_list['U']
V = stokes_list['V']

In [ ]:
# Constants
kB = 1.3806488e-16 # [erg K-1]
h = 6.6260755e-27  # [erg s]
c = 2.99792458e10  # [cm · s−1]
lambda0  = [6301.51*1e-8, 6302.50*1e-8]   # Angstroms to cm
l = np.mean(lambda0)
Icont = I.data_n[:,:,:5].mean(axis=2) # I map in continuum
Teff = 5780 # [K] T quiet sun average

In [ ]:
plot_data(Icont, title="Continuum Intensity")

In [ ]:
T = (1/Teff - kB*l/h/c * np.log(Icont))**-1

In [ ]:
plot_data(T, title="Temperature (Wien approximation)")

In [ ]:
M = 55.845 # Fe atomic mass, [g mol-1]
av = 6.022e23 # avogadro, [mol-1]
m =  M/av
Xi = 0 # microturbulence, assumed 0

kB = 1.3806488e-16 # [erg K-1] = [g cm^2 s^-2 K^-1]
c = 2.99792458e10  # [cm · s−1]

# Rebecca Centeno, equation 3
dlD = 6302/c * np.sqrt(2*kB*T/m + Xi**2) # will be in units of lambda, in this case Angstrom

plot_data(dlD, title=r"$\Delta \lambda _D$, in $\mathrm{\AA}$")

In [ ]:
# Umbra pixel
x_pix = 300
y_pix = 350

spectrum = copy.copy(V.data_n[y_pix,x_pix,:])
peaks_p, _ = find_peaks(spectrum, height=0.03)
peaks_n, _ = find_peaks(-spectrum, height=0.03)
print(f'Peaks positive: {stokes_list['V'].wave_array[peaks_p]}')
print(f'Peaks negative: {stokes_list['V'].wave_array[peaks_n]}')

plt.plot(stokes_list['V'].wave_array, stokes_list['V'].data_n[y_pix,x_pix,:], linestyle='-')
plt.vlines(stokes_list['V'].wave_array[peaks_p], ymin=-0.15, ymax=0.15, color='r')
plt.vlines(stokes_list['V'].wave_array[peaks_n], ymin=-0.15, ymax=0.15, color='g')

dlB = stokes_list['V'].wave_array[peaks_n] - stokes_list['V'].wave_array[peaks_p]
print(f'Distances: {dlB} Angstrom')

In [ ]:
gbar     = [1.669, 2.487] # Lozitsky

factor = gbar * dlB/dlD.max()

print(f'WFA factor: {factor}')


In [ ]:
x_pix = np.array([500, 180, 290, 400, 320, 300])
y_pix = np.array([118, 113, 538, 354, 428, 413])

# Plot profiles of each pixel
pixel_titles = [f'Quiet sun', f'Exterior to  sunspot', f'Penumbra 1', f'Penmbra 2', f'Umbra 1', f'Umbra 2']
fig, axs = plt.subplots(1,1, figsize=(12,7))
fig.suptitle('Profile variation for selected pixels', fontsize=20)

# Choose a colormap
num_lines = 6
cmap = plt.get_cmap('turbo')
colors = [cmap(i / num_lines) for i in range(num_lines)]

for j in range(num_lines):
    axs.plot(stokes_list['V'].wave_array, stokes_list['V'].data_n[y_pix[j],x_pix[j],:], linestyle='-', label=pixel_titles[j], color=colors[j])
    axs.set_title('V')
    # axs[row, col].vlines(line_wavelengths, stokes_list[param].data_n[y_pix[j],x_pix[j],:].min(), stokes_list[param].data_n[y_pix[j],x_pix[j],:].max(), colors='gray', linestyles='dashed')
    # axs[i, j].legend()


axs.vlines(stokes_list['V'].wave_array[peaks_p], ymin=-0.35, ymax=0.35, color='r')
axs.vlines(stokes_list['V'].wave_array[peaks_n], ymin=-0.35, ymax=0.35, color='g')

axs.legend()


In [ ]:
# Penumbra pixel
x_pix = 400
y_pix = 500

spectrum = copy.copy(V.data_n[y_pix,x_pix,:])
peaks_p, _ = find_peaks(spectrum, height=0.02)
peaks_n, _ = find_peaks(-spectrum, height=0.010)
print(f'Peaks positive: {stokes_list['V'].wave_array[peaks_p]}')
print(f'Peaks negative: {stokes_list['V'].wave_array[peaks_n]}')

plt.plot(stokes_list['V'].wave_array, stokes_list['V'].data_n[y_pix,x_pix,:], linestyle='-')
plt.vlines(stokes_list['V'].wave_array[peaks_p], ymin=-0.05, ymax=0.05, color='r')
plt.vlines(stokes_list['V'].wave_array[peaks_n], ymin=-0.05, ymax=0.05, color='g')

dlB = stokes_list['V'].wave_array[peaks_n] - stokes_list['V'].wave_array[peaks_p]
print(f'Distances: {-dlB} Angstrom')